In [141]:
import matplotlib.pyplot as plt
import json
import os
from PIL import Image
from sklearn.model_selection import train_test_split

import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import transforms
from torchvision.utils import make_grid

print('PyTorch version:', torch.__version__)
print('torchvision version:', torchvision.__version__)
use_gpu = torch.cuda.is_available()
print('Is GPU available:', use_gpu)

PyTorch version: 1.0.0
torchvision version: 0.2.1
Is GPU available: True


In [142]:
# general settings

# device
device = torch.device('cuda' if use_gpu else 'cpu')

# batchsize
batchsize = 128

# seed setting (warning : cuDNN's randomness is remaining)
seed = 1
torch.manual_seed(seed)
if use_gpu:
    torch.cuda.manual_seed(seed)

In [143]:
# directory settings
root_dir = '../../data/'
# directory for training data images
image_dir = root_dir + 'kkanji2_expansion_can_get_radical/'
# directory for save logs and trained weights
save_dir = root_dir + 'kkanji2_result'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

# load json for label
with open(root_dir + 'kanjivg_radical/utf16_to_radical.json') as f:
    utf16_to_radical = json.load(f)

In [144]:
# prepare dict for one-hot encoding
radical_set = set()
for value in utf16_to_radical.values():
    for v in value:
        radical_set.add(v)
radical_list = sorted(list(radical_set))

radical_dict = {}
for index, radical in enumerate(radical_list):
    radical_dict[radical] = index

n_radical = len(radical_dict)
print('the number of radical:', n_radical)

the number of radical: 1300


In [145]:
# make dataset class for image loading and label setting
class KanjiRadicalDataset(Dataset):
    def __init__(self, image_dir, image_name_list, utf16_to_radical, radical_dict, transform=None):
        self.image_dir = image_dir
        self.image_name_list = sorted(image_name_list)
        
        self.utf16_to_radical = utf16_to_radical
        self.radical_dict = radical_dict
        
        self.n_radical = len(radical_dict)
        self.transform = transform
        
    def __len__(self):
        return len(self.image_name_list)
    
    def __getitem__(self, idx):
        image_name = self.image_name_list[idx]        
        image = Image.open(self.image_dir + image_name)
        if self.transform:
            image = self.transform(image)
            
        label = torch.zeros(self.n_radical)
        utf16_code = image_name[:4]
        radical_list = self.utf16_to_radical[utf16_code]
        for radical in radical_list:
            label[radical_dict[radical]] = 1
            
        return image, label

In [165]:
# make dataset and train test split
train_name_list, validation_name_list = train_test_split(os.listdir(image_dir), test_size = 0.2, random_state = seed)

tf_train = transforms.Compose([transforms.RandomCrop(64, padding=8), transforms.ToTensor()])
tf_validation = transforms.ToTensor()

train_data = KanjiRadicalDataset(image_dir, train_name_list, utf16_to_radical, radical_dict, transform=tf_train)
validation_data = KanjiRadicalDataset(image_dir, validation_name_list, utf16_to_radical, radical_dict, transform=tf_validation)

print('The number of training data:', len(train_data))
print('The number of validation data:', len(validation_data))

# make DataLoader
train_loader = DataLoader(train_data, batch_size=batchsize, shuffle=True)
validation_loader = DataLoader(validation_data, batch_size=batchsize, shuffle=False)

The number of training data: 94127
The number of validation data: 23532


In [173]:
# データのラベルの分布を見る
# radical_count = torch.zeros(n_radical)
# for index, (image, label) in enumerate(validation_loader):
#    radical_count += torch.sum(label, dim=0)
#    print('\rprogress[%d/%d]' % (index+1, len(validation_loader)), end='')

In [174]:
# radical_count_non_zero = radical_count[radical_count > 0]
# plt.hist(radical_count_non_zero, bins = 30)
# plt.show()

In [175]:
# plt.bar(range(len(radical_count_non_zero)), radical_count_non_zero)
# plt.show()

In [178]:
# https://github.com/kuangliu/pytorch-cifar/blob/master/models/preact_resnet.pyより PreActResNet-18
# 本当は多分初期化をちゃんとやったほうが良いが、取り敢えずはこのまま
# preactResNet-18の採用理由はkmnistの提案論文のbaselineに合わせるためだが、あれは32x32のkmnist, k49で使われたものなので、
# 64x64のkkanjiに適用するのは微妙かも
# データセットの提案論文ではmanifold mixupをdata augumentationとして採用していますが、取り敢えずはまだやっていない

class PreActBlock(nn.Module):
    '''Pre-activation version of the BasicBlock.'''
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(PreActBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)

        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False)
            )

    def forward(self, x):
        out = F.relu(self.bn1(x))
        shortcut = self.shortcut(out) if hasattr(self, 'shortcut') else x
        out = self.conv1(out)
        out = self.conv2(F.relu(self.bn2(out)))
        out += shortcut
        return out

    
class PreActResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(PreActResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

# Todo:情報量ボトルネックがavg_poolの辺りにあるので、ImageNetのPreActResNetとかを参考に改善すること
def PreActResNet18(num_classes):
    return PreActResNet(PreActBlock, [2,2,2,2], num_classes)

In [177]:
criterion = nn.Sigmoid_Cross

AttributeError: module 'torch.nn.functional' has no attribute 'sigmoid_cross_entropy'

In [163]:
image[0][0]

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])